In [49]:
import os
import csv
from google.cloud import bigquery
from google.oauth2 import service_account

# this creates a path to where the source (large, cleaned) files reside locally
path_to_files = "//Users//krusty//Downloads//clean-files//"
# path_to_files = "//Users//krusty//Downloads//201701//"
clean_wedge_files = os.listdir(path_to_files) # assigning  to object 'clean_wedge_files'

In [51]:
len(clean_wedge_files)

53

In [52]:
service_path = "/Users/krusty/Desktop/MSBA /GBQ Key/"
service_file = 'my-project-366122-3a6c8a47fb52.json' # change this to your authentication information  
gbq_proj_id = 'my-project-366122' # change this to your project. 
gbq_dataset_id = 'wedge'
# And this should stay the same. 
private_key =service_path + service_file

In [53]:
# Get your credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And create a client to talk to GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [54]:
my_table = "wedge-366122-3a6c8a47fb52"

table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,my_table])

In [55]:
table_full_name

'my-project-366122.wedge.wedge-366122-3a6c8a47fb52'

In [56]:
def tbl_exists(client, table_ref):
    from google.cloud.exceptions import NotFound
    try:
        client.get_table(table_ref)
        return True
    except NotFound:
        return False

In [57]:
if not tbl_exists(client, table_full_name) :
    table_ref = client.create_table(
        table = table_full_name
    )
else :
    table_ref = client.get_table(table_full_name)

In [58]:
table = client.get_table(table_ref)
print("Table {} contains {} columns".format(table_ref.table_id,len(table.schema)))

Table wedge-366122-3a6c8a47fb52 contains 0 columns


In [59]:
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
job_config.schema_update_options = [
    bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION # This allows us to modify the table. 
]

In [74]:
job_config.schema = [
    bigquery.SchemaField("datetime", "STRING", mode="NULLABLE"), #changed to reflect error message 
    bigquery.SchemaField("register_no", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("emp_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("upc", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_subtype", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_status", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("department", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("quantity", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("Scale", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("cost", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("unitPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("total", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("regPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("altPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tax", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("taxexempt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("foodstamp", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("wicable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discountable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discounttype", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("voided", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("percentDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("ItemQtty", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volDiscType", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volume", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("VolSpecial", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("mixMatch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("matched", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memType", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("staff", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("numflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("itemstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tenderstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("charflag", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("varflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("batchHeaderID", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("local", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("organic", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("display", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("receipt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("card_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("store", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("branch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("match_id", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_id", "FLOAT", mode="NULLABLE"),
]
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 0

In [75]:
from timeit import default_timer as timer # to establish access to 'process timer'
start = timer()  # begin timer for this code block (will return #seconds elapsed)

for file in clean_wedge_files:
    my_table, junk = file.split(".")
    table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,my_table])
    if not tbl_exists(client, table_full_name) :
        table_ref =client.create_table(
            table = table_full_name
        )
    else :
        table_ref = client.get_table(table_full_name)
    table = client.get_table(table_ref)
    print("Table {} now contains {} columns.".format(table_ref.table_id, len(table.schema)))

    with open(path_to_files + file, "rb") as source_file:
        job = client.load_table_from_file(
            source_file,
            table_ref,
            location="US",  # Must match the destination dataset location.
            job_config=job_config,
        )  # API request

    job.result()  # Waits for table load to complete.
    print(
        "Loaded {} rows into {}:{}.".format(
            job.output_rows, 'wedge_example', table_ref.table_id
        )
    ) 

end = timer() # stops the code-block timer process after all iterations of all loops
print(end - start)  # displays total seconds elapsed in this code block process    

Table transArchive_201210_201212_inactive_clean now contains 0 columns.


BadRequest: 400 Error while reading data, error message: Could not parse 'register_no' as INT64 for field register_no (position 1) starting at location 0  with message 'Unable to parse'

In [13]:
# Checks the updated length of the schema
table = client.get_table(table)
print("Table {} now contains {} columns.".format(table_ref.table_id, len(table.schema)))

Table AtransArchive_201106_small now contains 50 columns.
